In [1]:
import pandas as pd
import psycopg2
import requests
import json
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None) 
from sklearn import preprocessing
import datetime as dt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [2]:
conn = psycopg2.connect(database="mid_term_project", user="lhl_student", password="lhl_student", host="mid-term-project.ca2jkepgjpne.us-east-2.rds.amazonaws.com", port="5432")
print("Database opened successfully")

Database opened successfully


In [3]:
df = pd.read_sql_query('''SELECT * FROM flights_test ORDER BY random() LIMIT 10000;''', conn)

In [4]:
df

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance
0,2020-01-01,B6,B6,B6,2952,B6,N633JB,2952,13204,MCO,"Orlando, FL",10721,BOS,"Boston, MA",1134,1425,N,171,1,1121
1,2020-01-27,UA,UA_CODESHARE,UA,5451,OO,N918SW,5451,14679,SAN,"San Diego, CA",12892,LAX,"Los Angeles, CA",1415,1520,N,65,1,109
2,2020-01-02,DL,DL_CODESHARE,DL,3526,OO,N8933B,3526,11898,GFK,"Grand Forks, ND",13487,MSP,"Minneapolis, MN",1520,1649,N,89,1,284
3,2020-01-09,DL,DL_CODESHARE,DL,3634,OO,N910EV,3634,12945,LEX,"Lexington, KY",11433,DTW,"Detroit, MI",1008,1136,N,88,1,296
4,2020-01-16,AS,AS_CODESHARE,AS,2811,QX,N440QX,2811,11884,GEG,"Spokane, WA",14057,PDX,"Portland, OR",1745,1908,N,83,1,279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2020-01-29,DL,DL_CODESHARE,DL,4217,OO,N426SW,4217,12397,ITH,"Ithaca/Cortland, NY",11433,DTW,"Detroit, MI",550,747,N,117,1,353
9996,2020-01-28,AA,AA_CODESHARE,AA,3289,OO,N611SK,3289,11298,DFW,"Dallas/Fort Worth, TX",14696,SBN,"South Bend, IN",1124,1444,N,140,1,846
9997,2020-01-18,AS,AS_CODESHARE,AS,2065,QX,N446QX,2065,12156,HLN,"Helena, MT",14747,SEA,"Seattle, WA",1535,1642,N,127,1,491
9998,2020-01-01,B6,B6,B6,2743,B6,N524JB,2743,11618,EWR,"Newark, NJ",14027,PBI,"West Palm Beach/Palm Beach, FL",1233,1529,N,176,1,1023


In [7]:
data = pd.read_csv('test_data.csv')

In [8]:
scaler = preprocessing.StandardScaler()
X = data.drop(columns='arr_delay').values
y = data['arr_delay'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

# Standardize the data using only statistics from the training set
# (always assume no knowledge of the test set for most unbiased performance estimate)
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [10]:
X_new = data.drop(columns='arr_delay').values

In [11]:
y_pred_linreg_new = reg.predict(X_new)

In [9]:
reg = LinearRegression()
k_folds = 5

# Cross-validation
cv_r2 = cross_val_score(reg, X_train, y_train, cv=k_folds, scoring='r2')

# Train model on the entire training set
reg.fit(X_train, y_train)

# Test the model
y_pred_linreg = reg.predict(X_test)

# Calculate metrics
r2_linreg = r2_score(y_test, y_pred_linreg)
rmse_linreg = mean_squared_error(y_test, y_pred_linreg, squared=False)

# Compare traing to test accuracy
linreg_training_accuracy = reg.score(X_train, y_train)
linreg_test_accuracy = reg.score(X_test, y_test)

# Print results
print('Linear Regression')
print('==================')
print(f'Cross-validated R^2\nMean: {cv_r2.mean():.4f}\nStd.: {cv_r2.std():.4f}')
print(f'R^2: {r2_linreg:.4f}')
print(f'RMSE: {rmse_linreg:.4f}')
print(f'Training accuracy: {linreg_training_accuracy:.4f}')
print(f'Test accuracy: {linreg_test_accuracy:.4f}')

Linear Regression
Cross-validated R^2
Mean: 1.0000
Std.: 0.0000
R^2: 1.0000
RMSE: 0.0000
Training accuracy: 1.0000
Test accuracy: 1.0000


In [17]:
print(r2_linreg)

1.0


In [18]:
y_pred_linreg_new

array([ -455.38717705,  -845.08822526,   -73.23609219, ...,
        -130.44535783, -1409.06457875,  -518.01671196])